In [77]:
import numpy as np
from scipy.interpolate import CubicSpline
import camb
from camb import model

class PowerSpectrumCalculator:
    def __init__(self, H0, Om):
        self.H0 = H0  # Hubble parameter
        self.Om = Om  # Total matter density parameter

    def _initialize_camb_params(self):
        pars = camb.CAMBparams()
        h = (self.H0 / 100)
        ombh2 = 0.045 * h**2  # Assuming baryon fraction is 0.045
        omch2 = (self.Om - 0.045) * h**2
        pars.set_cosmology(H0=self.H0, ombh2=ombh2, omch2=omch2)
        pars.InitPower.set_params(As=2.1e-9, ns=0.96)
        pars.set_matter_power(redshifts=[0], kmax=100.0)
        pars.NonLinear = model.NonLinear_none
        return pars

    def calculate_and_save_power_spectrum(self, filename):
        """Calculate the matter power spectrum and save it to a file."""
        camb_params = self._initialize_camb_params()
        results = camb.get_results(camb_params)
        kh, _, pk = results.get_linear_matter_power_spectrum(hubble_units=True, have_power_spectra=True, nonlinear=False)
        
        # Save to file
        np.savetxt(filename, np.column_stack([kh, pk[0]]), header="k [h/Mpc], P(k) [(Mpc/h)^3]", comments="#")
    
    
    def load_and_spline_power_spectrum(self, k, filename):
        """Load the power spectrum from a file and create a spline interpolator."""
        data = np.loadtxt(filename)
        kh = data[:, 0]  # k values
        pk = data[:, 1]  # P(k) values
        spline = CubicSpline(kh, pk, bc_type='natural')
        return spline
